# 104人力銀行職缺爬蟲

### 匯入所有所需套件

In [1]:
import pandas as pd
import json
import re
import requests
import time
from time import sleep
import csv
import random
from bs4 import BeautifulSoup as bs

## 爬取單一分頁內容

### 設定欲爬取之網址及內容

In [2]:
search_url = "https://www.104.com.tw/jobs/search/?cat=2007000000&jobsource=2018indexpoc&ro=0"
head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4'} #模擬真人查詢
# 使用 beautifulsoup4解析來自requests的search_url文檔
res = requests.get(search_url, headers = head)
print(res.encoding)
res.encoding = 'utf8'
soup = bs(res.text)

UTF-8


In [3]:
#搜尋所有職缺
jobs = soup.find_all('article',class_= 'js-job-item')
jobs

[<article class="b-block--top-bord job-list-item b-clearfix js-job-item js-job-item--focus b-block--ad" data-cust-name="智鈦星平台服務有限公司" data-cust-no="130000000131159" data-indcat="" data-indcat-desc="" data-is-apply="0" data-is-save="0" data-job-name="【Android 開發工程師】  Android Developer" data-job-no="11865514" data-job-ro="1" data-jobsource="hotjob_chr" data-qa-id="jobSeachResult">
 <div class="b-block__left">
 <h2 class="b-tit">
 <span class="b-tit__date"> <svg class="b-icon--gray b-icon--w18">
 <use xlink:href="//www.104.com.tw/jobs/search/static/img/sprite.svg#icon-focus"></use>
 </svg> </span>
 <a class="js-job-link" data-qa-id="jobSeachResultTitle" href="//www.104.com.tw/job/72bhm?jobsource=hotjob_chr" target="_blank">【Android 開發工程師】  Android Developer</a>
 </h2>
 <ul class="b-list-inline b-clearfix">
 <li class="b-list-inline__ad-icon"> </li>
 <li>
 <a href="//www.104.com.tw/company/1a2x6bksif?jobsource=hotjob_chr" target="_blank" title="公司名：智鈦星平台服務有限公司
 公司住址：台北市信義區市民大道六段288號8樓之8">智鈦

In [4]:
for job in jobs:
    job_name = job.find('a',class_='js-job-link').text
    job_company = job.get('data-cust-name')
    job_loc = job.find('ul',class_='job-list-intro').find('li').text
    job_pay = job.find('span',class_='b-tag--default').text
    job_url = job.find("a").get('href')
    
    print(" 職缺內容:",job_name,"\n","公司名稱:",job_company,"\n","地址:",job_loc,"\n","薪資:",job_pay,"\n","網址:",job_url,"\n",'-'*70)

 職缺內容: 【Android 開發工程師】  Android Developer 
 公司名稱: 智鈦星平台服務有限公司 
 地址: 台北市信義區 
 薪資: 待遇面議 
 網址: //www.104.com.tw/job/72bhm?jobsource=hotjob_chr 
 ----------------------------------------------------------------------
 職缺內容: ★年後報到★ 資安工程師(仁德廠) 
 公司名稱: 奇美實業股份有限公司 
 地址: 台南市仁德區 
 薪資: 待遇面議 
 網址: //www.104.com.tw/job/7l286?jobsource=hotjob_chr 
 ----------------------------------------------------------------------
 職缺內容: 【台中】iOS App Developer / App軟體開發工程師(台中市政府旁) 
 公司名稱: 英屬維京群島商瑞嘉耐思科技有限公司台灣分公司 
 地址: 台中市西屯區 
 薪資: 月薪57,750元以上 
 網址: //www.104.com.tw/job/6yqe6?jobsource=hotjob_chr 
 ----------------------------------------------------------------------
 職缺內容: 資訊設備專員 
 公司名稱: 太古汽車集團_英屬維京群島商太古國際汽車股份有限公司台灣分公司 
 地址: 台北市內湖區 
 薪資: 待遇面議 
 網址: //www.104.com.tw/job/7o0yy?jobsource=hotjob_chr 
 ----------------------------------------------------------------------
 職缺內容: 大流量社群平台資深網頁前端工程師 / Senior Web Engineer/Senior Frontend Engineer(React.js) 
 公司名稱: 宇泰華科技股份有限公司 
 地址: 台北市南港區 
 薪資: 年薪800,000~1,800,000元 
 網址: /

### 資料存成csv檔
#### 兩種方法都不用先在資料夾中設好檔案位置

### 方法一: 透過pandas套件儲存

In [5]:
content=[]

for job in jobs:
    job_name = job.find('a',class_='js-job-link').text
    job_company = job.get('data-cust-name')
    job_loc = job.find('ul',class_='job-list-intro').find('li').text
    job_pay = job.find('span',class_='b-tag--default').text
    job_url = job.find("a").get('href')
    job_data = {"職缺內容":job_name,"公司名稱":job_company,"地址":job_loc,"薪資":job_pay,"網址":job_url}
    content.append(job_data)
# 資料輸出
df = pd.DataFrame(content)
df.to_csv('104職缺內容(方法一).csv',encoding='big5')

### 方法二: 透過csv套件儲存

In [6]:
fn = "104職缺內容.csv" #取csv檔名
columns_name = ["職缺內容","公司名稱","地址","薪資","網址"] #第一欄的名稱
with open(fn,"w",newline="",) as csvFile: #定義csv的寫入檔，每次寫完會換行
    dictWriter = csv.DictWriter(csvFile,fieldnames=columns_name) #定義寫入器
    dictWriter.writeheader()
    for job in jobs:
        job_name = job.find('a',class_='js-job-link').text
        job_company = job.get('data-cust-name')
        job_loc = job.find('ul',class_='job-list-intro').find('li').text
        job_pay = job.find('span',class_='b-tag--default').text
        job_url = job.find("a").get('href')
        job_data = {"職缺內容":job_name,"公司名稱":job_company,"地址":job_loc,"薪資":job_pay,"網址":job_url}
        dictWriter.writerow(job_data)

### 資料存成json檔

In [7]:
fn = "104職缺內容.json"
obj = {}
obj["104職缺內容"] = []
for job in jobs:
    job_name = job.find('a',class_='js-job-link').text
    job_company = job.get('data-cust-name')
    job_loc = job.find('ul',class_='job-list-intro').find('li').text
    job_pay = job.find('span',class_='b-tag--default').text
    job_url = job.find("a").get('href')
    name = {"職缺內容": job_name,"公司名稱": job_company,"地址": job_loc,"薪資": job_pay,"網址": job_url}
    obj["104職缺內容"].append(name)
with open(fn, 'w', encoding='utf-8') as json_obj:
    json.dump(obj, json_obj,ensure_ascii=False)

## 爬取多個分頁內容 (104為AJAX動態載入網頁)
### 還在嘗試中@_@
### 難題: 回圈內網址雖然不一樣但抓的內容都是一樣的

### url_1+頁數+url_2=搜尋結果(link)

In [8]:
url_1 = "https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007000000&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=17&asc=0&page="
url_2 = "&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1"

all_job_datas=[]
for page in range(1,5+1):
    url = url_1+str(page)+url_2
    print(url)
    head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4'} 
    htmlFile = requests.get(url, headers = head)
    soups = bs(res.text)
    manyjobs = soups.find_all('article',class_= 'js-job-item')
    for job in manyjobs:
        job_name = job.find('a',class_='js-job-link').text
        job_company = job.get('data-cust-name')
        job_loc = job.find('ul',class_='job-list-intro').find('li').text
        job_pay = job.find('span',class_='b-tag--default').text
        job_url = job.find("a").get('href')
        job_data = {"職缺內容":job_name,"公司名稱":job_company,"地址":job_loc,"薪資":job_pay,"網址":job_url}
        all_job_datas.append(job_data)
        print(job_data)
    time.sleep(random.randint(1,3))

https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007000000&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=17&asc=0&page=1&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1
{'職缺內容': '【Android 開發工程師】  Android Developer', '公司名稱': '智鈦星平台服務有限公司', '地址': '台北市信義區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/72bhm?jobsource=hotjob_chr'}
{'職缺內容': '★年後報到★ 資安工程師(仁德廠)', '公司名稱': '奇美實業股份有限公司', '地址': '台南市仁德區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/7l286?jobsource=hotjob_chr'}
{'職缺內容': '【台中】iOS App Developer / App軟體開發工程師(台中市政府旁)', '公司名稱': '英屬維京群島商瑞嘉耐思科技有限公司台灣分公司', '地址': '台中市西屯區', '薪資': '月薪57,750元以上', '網址': '//www.104.com.tw/job/6yqe6?jobsource=hotjob_chr'}
{'職缺內容': '資訊設備專員', '公司名稱': '太古汽車集團_英屬維京群島商太古國際汽車股份有限公司台灣分公司', '地址': '台北市內湖區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/7o0yy?jobsource=hotjob_chr'}
{'職缺內容': '大流量社群平台資深網頁前端工程師 / Senior Web Engineer/Senior Frontend Engineer(React.js)', '公司名稱': '宇泰華科技股份有限公司', '地址': '台北市南港區', '薪資': '年薪800,000~1,800,000元', '網址': '/

https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007000000&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=17&asc=0&page=3&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1
{'職缺內容': '【Android 開發工程師】  Android Developer', '公司名稱': '智鈦星平台服務有限公司', '地址': '台北市信義區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/72bhm?jobsource=hotjob_chr'}
{'職缺內容': '★年後報到★ 資安工程師(仁德廠)', '公司名稱': '奇美實業股份有限公司', '地址': '台南市仁德區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/7l286?jobsource=hotjob_chr'}
{'職缺內容': '【台中】iOS App Developer / App軟體開發工程師(台中市政府旁)', '公司名稱': '英屬維京群島商瑞嘉耐思科技有限公司台灣分公司', '地址': '台中市西屯區', '薪資': '月薪57,750元以上', '網址': '//www.104.com.tw/job/6yqe6?jobsource=hotjob_chr'}
{'職缺內容': '資訊設備專員', '公司名稱': '太古汽車集團_英屬維京群島商太古國際汽車股份有限公司台灣分公司', '地址': '台北市內湖區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/7o0yy?jobsource=hotjob_chr'}
{'職缺內容': '大流量社群平台資深網頁前端工程師 / Senior Web Engineer/Senior Frontend Engineer(React.js)', '公司名稱': '宇泰華科技股份有限公司', '地址': '台北市南港區', '薪資': '年薪800,000~1,800,000元', '網址': '/

https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007000000&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=17&asc=0&page=5&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1
{'職缺內容': '【Android 開發工程師】  Android Developer', '公司名稱': '智鈦星平台服務有限公司', '地址': '台北市信義區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/72bhm?jobsource=hotjob_chr'}
{'職缺內容': '★年後報到★ 資安工程師(仁德廠)', '公司名稱': '奇美實業股份有限公司', '地址': '台南市仁德區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/7l286?jobsource=hotjob_chr'}
{'職缺內容': '【台中】iOS App Developer / App軟體開發工程師(台中市政府旁)', '公司名稱': '英屬維京群島商瑞嘉耐思科技有限公司台灣分公司', '地址': '台中市西屯區', '薪資': '月薪57,750元以上', '網址': '//www.104.com.tw/job/6yqe6?jobsource=hotjob_chr'}
{'職缺內容': '資訊設備專員', '公司名稱': '太古汽車集團_英屬維京群島商太古國際汽車股份有限公司台灣分公司', '地址': '台北市內湖區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/7o0yy?jobsource=hotjob_chr'}
{'職缺內容': '大流量社群平台資深網頁前端工程師 / Senior Web Engineer/Senior Frontend Engineer(React.js)', '公司名稱': '宇泰華科技股份有限公司', '地址': '台北市南港區', '薪資': '年薪800,000~1,800,000元', '網址': '/

### 資料存成csv檔

In [9]:
fn = "104職缺內容(多頁).csv"
columns_name = ["職缺內容","公司名稱","地址","薪資","網址"]
with open(fn,"w",newline="",encoding="utf_8_sig") as csvFile:
    dictWriter = csv.DictWriter(csvFile,fieldnames=columns_name,)
    dictWriter.writeheader()
    for data in all_job_datas:
        dictWriter.writerow(data)

### 資料存成json檔

In [10]:
fn = "104職缺內容(多頁).json"
obj = {}
obj["104職缺內容"] = []
for data in all_job_datas:
    name = {"職缺內容": job_name,"公司名稱": job_company,"地址": job_loc,"薪資": job_pay,"網址": job_url}
    obj["104職缺內容"].append(name)
with open(fn, 'w', encoding='utf-8') as json_obj:
    json.dump(obj, json_obj,ensure_ascii=False)